## READ DATA

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/co2-emission-by-vehicles/CO2 Emissions_Canada.csv
/kaggle/input/co2-emission-by-vehicles/Data Description.csv


In [2]:
%pip show numpy

Name: numpy
Version: 1.26.4
Summary: Fundamental package for array computing in Python
Home-page: https://www.numpy.org
Author: Travis E. Oliphant et al.
Author-email: 
License: BSD-3-Clause
Location: /usr/local/lib/python3.10/dist-packages
Requires: mkl, mkl-service, mkl_fft, mkl_random, mkl_umath, tbb4py
Required-by: accelerate, albucore, albumentations, arviz, astropy, autograd, bayesian-optimization, bigframes, blis, blosc2, bokeh, Boruta, Bottleneck, bqplot, Cartopy, catboost, category_encoders, cesium, chex, clarabel, cmdstanpy, contourpy, cudf-cu12, cufflinks, cuml-cu12, cupy-cuda12x, cuvs-cu12, cvxpy, dask-cuda, dask-cudf-cu12, datascience, datasets, datashader, db-dtypes, deap, diffusers, dipy, dopamine_rl, easyocr, eli5, fasttext, featuretools, flax, folium, fury, geemap, gensim, geopandas, gym, gymnasium, h5py, hep_ml, holoviews, hyperopt, ibis-framework, ImageHash, imageio, imbalanced-learn, imgaug, ipympl, jax, jaxlib, kaggle-environments, keras, keras-core, keras-hub, lan

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
import pandas as pd
import seaborn as sns

In [4]:
df = pd.read_csv("/kaggle/input/co2-emission-by-vehicles/CO2 Emissions_Canada.csv")
df.shape
df.head(6)

,Make,Model,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City (L/100 km),Fuel Consumption Hwy (L/100 km),Fuel Consumption Comb (L/100 km),Fuel Consumption Comb (mpg),CO2 Emissions(g/km)
0,ACURA,ILX,COMPACT,2.0,4,AS5,Z,9.9,6.7,8.5,33,196
1,ACURA,ILX,COMPACT,2.4,4,M6,Z,11.2,7.7,9.6,29,221
2,ACURA,ILX HYBRID,COMPACT,1.5,4,AV7,Z,6.0,5.8,5.9,48,136
3,ACURA,MDX 4WD,SUV - SMALL,3.5,6,AS6,Z,12.7,9.1,11.1,25,255
4,ACURA,RDX AWD,SUV - SMALL,3.5,6,AS6,Z,12.1,8.7,10.6,27,244
5,ACURA,RLX,MID-SIZE,3.5,6,AS6,Z,11.9,7.7,10.0,28,230


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7385 entries, 0 to 7384
Data columns (total 12 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Make                              7385 non-null   object 
 1   Model                             7385 non-null   object 
 2   Vehicle Class                     7385 non-null   object 
 3   Engine Size(L)                    7385 non-null   float64
 4   Cylinders                         7385 non-null   int64  
 5   Transmission                      7385 non-null   object 
 6   Fuel Type                         7385 non-null   object 
 7   Fuel Consumption City (L/100 km)  7385 non-null   float64
 8   Fuel Consumption Hwy (L/100 km)   7385 non-null   float64
 9   Fuel Consumption Comb (L/100 km)  7385 non-null   float64
 10  Fuel Consumption Comb (mpg)       7385 non-null   int64  
 11  CO2 Emissions(g/km)               7385 non-null   int64  
dtypes: flo

## PREPROCESSING

In [6]:
# Drop unnecessary columns
df = df.drop(['Fuel Consumption City (L/100 km)', 'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'], axis = 1)
df.head(6)

,Make,Model,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption Comb (mpg),CO2 Emissions(g/km)
0,ACURA,ILX,COMPACT,2.0,4,AS5,Z,33,196
1,ACURA,ILX,COMPACT,2.4,4,M6,Z,29,221
2,ACURA,ILX HYBRID,COMPACT,1.5,4,AV7,Z,48,136
3,ACURA,MDX 4WD,SUV - SMALL,3.5,6,AS6,Z,25,255
4,ACURA,RDX AWD,SUV - SMALL,3.5,6,AS6,Z,27,244
5,ACURA,RLX,MID-SIZE,3.5,6,AS6,Z,28,230


In [7]:
# Rename the columns
df.rename(columns={ 'Make': 'make',
                    'Model': 'model',
                    'Vehicle Class': 'vehicle_class',
                    'Engine Size(L)': 'engine_size',
                    'Cylinders': 'cylinders',
                    'Transmission': 'transmission',
                    'Fuel Type': 'fuel_type',
                    'Fuel Consumption Comb (mpg)': 'fuel_consumption',
                    'CO2 Emissions(g/km)': 'co2'
                    }, inplace=True)
df.head(6)

,make,model,vehicle_class,engine_size,cylinders,transmission,fuel_type,fuel_consumption,co2
0,ACURA,ILX,COMPACT,2.0,4,AS5,Z,33,196
1,ACURA,ILX,COMPACT,2.4,4,M6,Z,29,221
2,ACURA,ILX HYBRID,COMPACT,1.5,4,AV7,Z,48,136
3,ACURA,MDX 4WD,SUV - SMALL,3.5,6,AS6,Z,25,255
4,ACURA,RDX AWD,SUV - SMALL,3.5,6,AS6,Z,27,244
5,ACURA,RLX,MID-SIZE,3.5,6,AS6,Z,28,230


In [8]:
# Check missing data - Luckily, no missing data
df_recommender = df.copy()
df.isnull().sum()

make                0
model               0
vehicle_class       0
engine_size         0
cylinders           0
transmission        0
fuel_type           0
fuel_consumption    0
co2                 0
dtype: int64

In [9]:
df_recommender.loc[[2049]]

,make,model,vehicle_class,engine_size,cylinders,transmission,fuel_type,fuel_consumption,co2
2049,RAM,1500 4X4,PICKUP TRUCK - STANDARD,3.6,6,A8,X,22,290


### Categorical features preprocessing

**Encode "make" column**

In [10]:
df["make"].unique()

array(['ACURA', 'ALFA ROMEO', 'ASTON MARTIN', 'AUDI', 'BENTLEY', 'BMW',
       'BUICK', 'CADILLAC', 'CHEVROLET', 'CHRYSLER', 'DODGE', 'FIAT',
       'FORD', 'GMC', 'HONDA', 'HYUNDAI', 'INFINITI', 'JAGUAR', 'JEEP',
       'KIA', 'LAMBORGHINI', 'LAND ROVER', 'LEXUS', 'LINCOLN', 'MASERATI',
       'MAZDA', 'MERCEDES-BENZ', 'MINI', 'MITSUBISHI', 'NISSAN',
       'PORSCHE', 'RAM', 'ROLLS-ROYCE', 'SCION', 'SMART', 'SRT', 'SUBARU',
       'TOYOTA', 'VOLKSWAGEN', 'VOLVO', 'GENESIS', 'BUGATTI'],
      dtype=object)

In [11]:
from sklearn.preprocessing import LabelEncoder

# Create a LabelEncoder object
le = LabelEncoder()

# Fit and transform the categorical data
df['make'] = le.fit_transform(df['make'])
df["make"].unique()

array([ 0,  1,  2,  3,  4,  5,  7,  8,  9, 10, 11, 12, 13, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 14,  6])

**Encode "vehicle_class" column**

In [12]:
df["vehicle_class"].unique()

array(['COMPACT', 'SUV - SMALL', 'MID-SIZE', 'TWO-SEATER', 'MINICOMPACT',
       'SUBCOMPACT', 'FULL-SIZE', 'STATION WAGON - SMALL',
       'SUV - STANDARD', 'VAN - CARGO', 'VAN - PASSENGER',
       'PICKUP TRUCK - STANDARD', 'MINIVAN', 'SPECIAL PURPOSE VEHICLE',
       'STATION WAGON - MID-SIZE', 'PICKUP TRUCK - SMALL'], dtype=object)

In [13]:
df['vehicle_class'] = le.fit_transform(df['vehicle_class'])
df['vehicle_class'].unique()

array([ 0, 11,  2, 13,  3, 10,  1,  9, 12, 14, 15,  6,  4,  7,  8,  5])

**Encode "transmission" column**

In [14]:
df["transmission"].unique()

array(['AS5', 'M6', 'AV7', 'AS6', 'AM6', 'A6', 'AM7', 'AV8', 'AS8', 'A7',
       'A8', 'M7', 'A4', 'M5', 'AV', 'A5', 'AS7', 'A9', 'AS9', 'AV6',
       'AS4', 'AM5', 'AM8', 'AM9', 'AS10', 'A10', 'AV10'], dtype=object)

In [15]:
df['transmission'] = le.fit_transform(df['transmission'])
df['transmission'].unique()

array([14, 25, 22, 15,  8,  3,  9, 23, 17,  4,  5, 26,  1, 24, 19,  2, 16,
        6, 18, 21, 13,  7, 10, 11, 12,  0, 20])

**Encode "fuel_type" column**

In [16]:
df["fuel_type"].unique()

array(['Z', 'D', 'X', 'E', 'N'], dtype=object)

In [17]:
df['fuel_type'] = le.fit_transform(df['fuel_type'])
df['fuel_type'].unique()

array([4, 0, 3, 1, 2])

**Encode "model" column**

In [18]:
print(df["model"].unique())
len(df["model"].unique())

['ILX' 'ILX HYBRID' 'MDX 4WD' ... 'Tacoma 4WD D-Cab TRD Off-Road/Pro'
 'Atlas Cross Sport 4MOTION' 'XC40 T4 AWD']


2053

In [19]:
# Calculate the frequency of each category
frequency = df['model'].value_counts()

# Replace values with frequency less than 4 with 'Other'
df['model'] = df['model'].apply(lambda x: x if frequency[x] >= 4 else 'Other')

len(df["model"].unique())

759

In [20]:
# Perform One-Hot Encoding
one_hot_encoded_df = pd.get_dummies(df, columns=['model'], prefix='model')
one_hot_encoded_df

,make,vehicle_class,engine_size,cylinders,transmission,fuel_type,fuel_consumption,co2,model_124 SPIDER,model_124 Spider,...,model_Yukon 4WD,model_Yukon 4WD FFV,model_Yukon FFV,model_Yukon XL 4WD,model_Yukon XL 4WD FFV,model_Yukon XL FFV,model_Z4 sDRIVE28i,model_Z4 sDRIVE35i,model_tC,model_xB
0,0,0,2.0,4,14,4,33,196,False,False,...,False,False,False,False,False,False,False,False,False,False
1,0,0,2.4,4,25,4,29,221,False,False,...,False,False,False,False,False,False,False,False,False,False
2,0,0,1.5,4,22,4,48,136,False,False,...,False,False,False,False,False,False,False,False,False,False
3,0,11,3.5,6,15,4,25,255,False,False,...,False,False,False,False,False,False,False,False,False,False
4,0,11,3.5,6,15,4,27,244,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7380,41,11,2.0,4,17,4,30,219,False,False,...,False,False,False,False,False,False,False,False,False,False
7381,41,11,2.0,4,17,4,29,232,False,False,...,False,False,False,False,False,False,False,False,False,False
7382,41,11,2.0,4,17,4,27,240,False,False,...,False,False,False,False,False,False,False,False,False,False
7383,41,12,2.0,4,17,4,29,232,False,False,...,False,False,False,False,False,False,False,False,False,False


## TRAINING - CO2 emission prediction

In [21]:
one_hot_encoded_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7385 entries, 0 to 7384
Columns: 767 entries, make to model_xB
dtypes: bool(759), float64(1), int64(7)
memory usage: 5.8 MB


### Data split

In [22]:
from sklearn.model_selection import train_test_split

X = one_hot_encoded_df.drop(columns=['co2'])  # All columns except 'co2'
y = one_hot_encoded_df['co2'] 

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=10)

### Train Linear Regression

In [24]:
from sklearn.linear_model import LinearRegression
from sklearn import metrics

lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

### Train Decision Tree

In [25]:
dt = DecisionTreeRegressor(random_state=42)
dt.fit(X_train, y_train)

DecisionTreeRegressor(random_state=42)

### Test

In [26]:
def print_evaluate(true, predicted):  
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    print('MAE:', mae)
    print('MSE:', mse)
    print('RMSE:', rmse)
    print('R2 Square', r2_square)
    print('__________________________________')
    
def evaluate(true, predicted):
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    return mae, mse, rmse, r2_square

In [27]:
# Evaluate Linear Regression
test_pred = lr.predict(X_test)
train_pred = lr.predict(X_train)
print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, test_pred)
print('Train set evaluation:\n_____________________________________')
print_evaluate(y_train, train_pred)

Test set evaluation:
_____________________________________
MAE: 9.580988550880503
MSE: 197.31241261077815
RMSE: 14.046793677233897
R2 Square 0.9439169981594613
__________________________________
Train set evaluation:
_____________________________________
MAE: 8.420551945591878
MSE: 163.1813624206575
RMSE: 12.774246060752764
R2 Square 0.9519954664378748
__________________________________


In [28]:
# Evaluate Decision Tree
test_pred = dt.predict(X_test)
train_pred = dt.predict(X_train)
print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, test_pred)
print('Train set evaluation:\n_____________________________________')
print_evaluate(y_train, train_pred)

Test set evaluation:
_____________________________________
MAE: 2.7712518110791637
MSE: 18.44533041997585
RMSE: 4.294802721892572
R2 Square 0.9947572000858685
__________________________________
Train set evaluation:
_____________________________________
MAE: 1.0129608543115651
MSE: 3.5352053517083983
RMSE: 1.8802141770841954
R2 Square 0.9989600167480057
__________________________________


In [29]:
X_test.iloc[[1]]

,make,vehicle_class,engine_size,cylinders,transmission,fuel_type,fuel_consumption,model_124 SPIDER,model_124 Spider,model_1500,...,model_Yukon 4WD,model_Yukon 4WD FFV,model_Yukon FFV,model_Yukon XL 4WD,model_Yukon XL 4WD FFV,model_Yukon XL FFV,model_Z4 sDRIVE28i,model_Z4 sDRIVE35i,model_tC,model_xB
2049,33,6,3.6,6,5,3,22,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [30]:
dt.predict(X_test.iloc[[1]])

array([297.33333333])

## TRAINING - Recommender by KMeans

In [31]:
df_recommender

,make,model,vehicle_class,engine_size,cylinders,transmission,fuel_type,fuel_consumption,co2
0,ACURA,ILX,COMPACT,2.0,4,AS5,Z,33,196
1,ACURA,ILX,COMPACT,2.4,4,M6,Z,29,221
2,ACURA,ILX HYBRID,COMPACT,1.5,4,AV7,Z,48,136
3,ACURA,MDX 4WD,SUV - SMALL,3.5,6,AS6,Z,25,255
4,ACURA,RDX AWD,SUV - SMALL,3.5,6,AS6,Z,27,244
...,...,...,...,...,...,...,...,...,...
7380,VOLVO,XC40 T5 AWD,SUV - SMALL,2.0,4,AS8,Z,30,219
7381,VOLVO,XC60 T5 AWD,SUV - SMALL,2.0,4,AS8,Z,29,232
7382,VOLVO,XC60 T6 AWD,SUV - SMALL,2.0,4,AS8,Z,27,240
7383,VOLVO,XC90 T5 AWD,SUV - STANDARD,2.0,4,AS8,Z,29,232


In [32]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import numpy as np

# Standardize the features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df_recommender[['engine_size', 'cylinders', 'fuel_consumption', 'co2']].values)

In [33]:
# Apply K-Means clustering
kmeans = KMeans(n_clusters=3, random_state=42, n_init='auto')
df_recommender['cluster'] = kmeans.fit_predict(scaled_features)
df_recommender.head(5)

,make,model,vehicle_class,engine_size,cylinders,transmission,fuel_type,fuel_consumption,co2,cluster
0,ACURA,ILX,COMPACT,2.0,4,AS5,Z,33,196,2
1,ACURA,ILX,COMPACT,2.4,4,M6,Z,29,221,2
2,ACURA,ILX HYBRID,COMPACT,1.5,4,AV7,Z,48,136,2
3,ACURA,MDX 4WD,SUV - SMALL,3.5,6,AS6,Z,25,255,1
4,ACURA,RDX AWD,SUV - SMALL,3.5,6,AS6,Z,27,244,1


In [34]:
# Recommendation function
def recommend_car(desired_engine_size, desired_cylinders, desired_fuel_consumption, desired_co2):
    # Scale the input features
    scaled_input = scaler.transform([[desired_engine_size, desired_cylinders, desired_fuel_consumption, desired_co2]])[0]
    
    # Predict the closest cluster
    cluster_centers = kmeans.cluster_centers_
    closest_cluster_idx = np.argmin(np.linalg.norm(cluster_centers - scaled_input, axis=1))
    
    # Filter cars in the closest cluster
    cluster_cars = df_recommender[df_recommender['cluster'] == closest_cluster_idx].copy()
    
    # Calculate distances to all cars in the cluster
    cluster_cars['distance'] = cluster_cars.apply(
        lambda row: np.linalg.norm(scaler.transform([[row['engine_size'], row['cylinders'], row['fuel_consumption'], row['co2']]])[0] - scaled_input),
        axis=1
    )
    
    # Recommend the car with the smallest distance
    recommended_cars = cluster_cars.nsmallest(8, 'distance')
    
    return recommended_cars

In [35]:
# Test the recommendation system
desired_engine_size = 2.0
desired_cylinders = 6
desired_fuel_consumption = 10.0
desired_co2 = 200

recommendation = recommend_car(desired_engine_size, desired_cylinders, desired_fuel_consumption, desired_co2)

# Print the recommendation
print(f"Recommended Cars:")
recommendation[["make", "model", "vehicle_class", "transmission", "fuel_type", "distance"]]

Recommended Cars:


,make,model,vehicle_class,transmission,fuel_type,distance
655,LAND ROVER,RANGE ROVER V6 3.0 SC FFV,SUV - STANDARD,AS8,E,1.722148
649,LAND ROVER,RANGE ROVER SPORT V6 3.0 SC FFV,SUV - STANDARD,AS8,E,1.735244
6797,FORD,F-150 FFV,PICKUP TRUCK - STANDARD,AS6,E,1.749921
5758,FORD,F-150 FFV,PICKUP TRUCK - STANDARD,AS6,E,1.765212
5760,FORD,F-150 FFV (LT Tire Pkg),PICKUP TRUCK - STANDARD,AS6,E,1.765212
1513,FORD,TAURUS FFV,FULL-SIZE,AS6,E,1.770297
4720,FORD,F-150 FFV,PICKUP TRUCK - STANDARD,AS6,E,1.773055
4723,FORD,F-150 FFV (LT Tire Pkg),PICKUP TRUCK - STANDARD,AS6,E,1.773055


## Export models

In [36]:
# import joblib

# # Save the models
# joblib.dump(lr, 'linear_regression_model.pkl')
# joblib.dump(dt, 'decision_tree_model.pkl')
# joblib.dump(kmeans, 'kmeans_model.pkl')

# print("Models saved successfully!")